# Self-Supervised Monocular Depth Estimation using Neural Network

## Abstract

#### The purpose of this project is to investigate the self-supervised monocular depth estimation problem. There are lots of traditional approaches which extracts disparity map from binocular, motion, and multi-view stereo, but most of them require multiple viewpoints of the scene or the geometric information of the scene to be available. The supervised learning networks also achieved great success, but they suffer from the problem of insufficent training data since the ground truth disparity is hard to obtain. Therefore, the self-supervised monocular despth estimation method was proposed where only one image of the scene is required at test time, and there is no need of the ground truth target. 

#### In this project, we followed one of the self-supervised single image depth estimation research and implemented the network based on the result from this paper: [Unsupervised Monocular Depth Estimation with Left-Right Consistency](https://arxiv.org/abs/1609.03677v3). The authors proposed a CNN to perform single image depth estimation, despite the absence of ground truth depth data. The main innovation from this paper is the training loss which aims to enforce the consistency between the predicted left and right disparities, which will be further discussed in the following sections. 

## Team members and contribution

#### Yiran Cao
* implemented the dataloader, train and test function
* visualization of the loss curve and resulting disparity maps
* debugging

#### Haoyu Tian
* contribution 1
* contribution 2

#### Tengyu Cai
* contribution 1
* contribution 2

# Code Libraries

- NumPy
- PyTorch
- Scikit-image
- Torchvision
- Matplotlib
- Python Imaging Library

In [2]:
from mylibs.dataloader import KittiDataset
from mylibs.transformation import ToResizeImage, ToRandomFlip, ToTensor, AugumentImagePair
from mylibs.model import Model

import torch
from torch.utils.data import DataLoader

import os
import numpy as np
import random
import skimage.transform
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torchvision.transforms.functional as tF

from skimage.metrics import structural_similarity as ssim

In [2]:
USE_GPU = False

## Data Loading

### Dataset

We used the [KITTI driving dataset](http://www.cvlibs.net/datasets/kitti/raw_data.php) as our data for training and testing. Only data in the "city" category (around 30G) are used in the experiment due to the limited resources on memory and GPU. "2011_09_26_drive_0002_sync" is dedicated for testing only, and the rest are used as the training data. (The uploaded zip file contains only "2011_09_26_drive_0048_sync", and "2011_09_26_drive_0002_sync" as the training and testing data, under the directory `./data/train`, `./data/test`)

In [3]:
train_path = os.path.join(os.getcwd(), 'data/train')
test_path = os.path.join(os.getcwd(), 'data/test')

### Transformation

In [4]:
train_transform = transforms.Compose([
    ToResizeImage(),
    ToRandomFlip(),
    ToTensor(),
    AugumentImagePair(),
])

test_transform = transforms.Compose([
    ToResizeImage(),
    ToTensor(),
])

### Data Loader

The `KittiDataset` class takes the path to train/test dataset and loads the left and right images (depending on whether it is used for training or testing) in to a dictionary, then applies the transformations defined above. There are four disparity maps with different scales produced for each sample. The full implementation is at `./mylibs/dataloader.py`

In [5]:
train_set = KittiDataset(train_path, 'train', transform = train_transform)
test_set = KittiDataset(test_path, 'test', transform = test_transform)

train_loader = DataLoader(train_set, batch_size = 5, shuffle = True)
test_loader = DataLoader(test_set, batch_size = 77, shuffle = False)

In [6]:
# for data in train_set:
#     left = data['left_image'][0]
#     right = data['right_image'][0]
#     ssim = ssim(left, right, data_range=right.max() - right.min(), multichannel=True)

## Implement CNN using PyTorch

The CNN has a encoder-decoder similar structure, which performs the image reconstruction. The objective is to train the model so that it can learn the disparity warp and use it to estimate the right image from the left image. Detailed information on our model architecture is presented in the figure below. <img src="images\model_architecture.png"> 

In [7]:
model = Model(train_loader=train_loader,
              test_loader=test_loader,
              device='cuda' if USE_GPU else 'cpu', 
              epochs=1, 
              save_per_epoch=10, 
              img_height=256, 
              img_width=512, 
              model_path='output_model', 
              disp_path='output_disp')

## Train the model and plot the loss curve

### Training Stage
During each epoch, only the left image is passed into the network. Then the output of the model along with the original left and right image are passed to the `Loss` class for computing the training loss. The Adam optimizer is used in the experiment with a learning rate 0.0001. The trained model will be saved to a `.pt` file every `save_per_epoch=10` epochs, so that it is easier continue the training process or evaluate the trained model.

### Loss Computation
The overall loss is calculated as the weighted sum of three components, including smoothness, reconstruction, and left-right disparity consistency. The loss is computed over all four different scales for each sample.

- Smoothness: The smoothness loss aims to avoid the unnecessary large disparity jumps in the prediction. We first compute the disparity gradient and then the L1 penalty on it. 

- Reconstruction: The reconstruction loss is calculated using a weighted sum of L1 and the structural similarity index measure (SSIM) metric. It is evaluated on the original left/right image and the predicted left/right image to measure the difference between two images. 

- Left-right Disparity Consistency: The left-right disparity consistency loss is to enforce that the predicted left and right disparity map should be identical. The L1 metric is used.

More details in `./mylibs/loss.py`

In [ ]:
loss = model.train()

In [ ]:
# plot the loss curve
loss = np.array(loss)

fig = plt.figure(figsize=(12,6)) 
plt.subplots_adjust(bottom=0.2,right=0.85,top=0.95) 
ax = fig.add_subplot(1,1,1)

ax.clear() 
ax.set_xlabel('iterations')
ax.set_ylabel('loss value') 
ax.set_title('Training loss curve') 
ax.plot(loss, label='training loss') 
ax.legend(loc='upper right') 
fig.canvas.draw() 

## Test the trained model on the testing dataset

### Testing Stage
Testing starts from loading the model specified in function parameter. Then, only the left image is passed to the network to predict the disparity map from the trained model. We saved all disparity maps with scale 256x512 as the `.npy` file, and we have also save one version with the post-processing applied. The post-processing ...

Finally, we visualize the original image, disparity, and the post-processed disparity in the following block. A random sample and the result will be displayed when running the cell.

In [ ]:
model.test('output_model/0.pt')

In [ ]:
# plot original image, disparity, and disparity with post-processing
test_size = len(test_loader.dataset)

for data in test_loader:
    idx = random.randint(0, len(data['left_image']))
    img = data['left_image'][idx].permute(1,2,0)

disp = np.load('output_disp/disparities.npy')
disp_pp = np.load('output_disp/disparities_pp.npy')

disp_to_img = skimage.transform.resize(disp[idx].squeeze(), [256, 512], mode='constant')
disp_pp_to_img = skimage.transform.resize(disp_pp[idx].squeeze(), [256, 512], mode='constant')

fig = plt.figure(figsize=(16, 12))

ax = fig.add_subplot(1,3,1)
plt.title('original sample ' + str(idx))
plt.imshow(img, cmap='plasma')

ax = fig.add_subplot(1,3,2)
plt.title('disparity without post-processing')
plt.imshow(disp_to_img, cmap='plasma')

ax = fig.add_subplot(1,3,3)
plt.title('disparity with post-processing')
plt.imshow(disp_pp_to_img, cmap='plasma')

## Disscussion/Conclusion ...

## Reference

- [Unsupervised Monocular Depth Estimation with Left-Right Consistency by Cle ́ment Godard, Oisin Mac Aodha and Gabriel J. Brostow](https://arxiv.org/abs/1609.03677v3) and its [Git Repo](https://github.com/mrharicot/monodepth)